In [1]:
import os
import joblib
import numpy as np
from model_settings import ms
from pathlib import Path
ms.find_root(Path())
models_dir = os.path.join(ms.root,ms.trained_models)
models = [f for f in os.listdir(models_dir) if f.find('.')==-1]
for i,m in enumerate(models):
    print(f"{i}  {m}")
i = int(input('select model: '))
model_dir = os.path.join(models_dir,models[i])
pickle = os.path.join(model_dir,models[i]+'.pkl')

def test_performance(model,net):
    inpred = net.predict(model['train_X'])
    intar = model['train_data'][model['target_name']].values
    indiff = inpred-intar
    inRMSE = np.sqrt(np.mean(indiff**2))
    inMAE = np.mean(np.abs(indiff))
    print(f"out of sample\n   RMSE:\n      {inRMSE}")
    print(f"   MAE:\n      {inMAE}")

    outpred = net.predict(model['test_X'])
    outtar = model['test_data'][model['target_name']].values
    outdiff = outpred-outtar
    outRMSE = np.sqrt(np.mean(outdiff**2))
    outMAE = np.mean(np.abs(outdiff))
    print(f"out of sample\n   RMSE:\n      {outRMSE}")
    print(f"   MAE:\n      {outMAE}")

0  2024-12-03 115353819604 inital cboe spx relative asian
1  2024-12-03 221708375538 inital cboe spx relative barrier
2  2024-12-04 001549051867 inital cboe spx relative asian
3  2024-12-04 011958990550 inital cboe spx relative asian
4  2024-12-04 022805473467 inital cboe spx relative asian


select model:  4


In [2]:
model = joblib.load(pickle)

In [3]:
net = model['fitted']
test_performance(model,net)

out of sample
   RMSE:
      0.005064355865830351
   MAE:
      0.0030570028051901126
out of sample
   RMSE:
      0.055429415545038066
   MAE:
      0.03542515603491769


In [4]:
model['mlp_params']

{'solver': 'sgd',
 'activation': 'relu',
 'max_iter': 500,
 'random_state': 1312,
 'early_stopping': True,
 'hidden_layer_sizes': (20,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.1,
 'n_iter_no_change': 20,
 'tol': 0.0001,
 'warm_start': False}

In [5]:
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
# model['mlp_params']['verbose']=True
mlp = MLPRegressor(**model['mlp_params'])
mlp.out_activation_= 'relu'
net = Pipeline([
            ("preprocessor", model['preprocessor']),
            ("regressor", mlp)
        ])
net[1].__dict__

{'activation': 'relu',
 'solver': 'sgd',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.1,
 'power_t': 0.5,
 'max_iter': 500,
 'loss': 'squared_error',
 'hidden_layer_sizes': (20,),
 'shuffle': True,
 'random_state': 1312,
 'tol': 0.0001,
 'verbose': False,
 'warm_start': False,
 'momentum': 0.9,
 'nesterovs_momentum': True,
 'early_stopping': True,
 'validation_fraction': 0.1,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-08,
 'n_iter_no_change': 20,
 'max_fun': 15000,
 'out_activation_': 'relu'}

In [6]:
net.fit(model['train_X'],model['train_y'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  ['days_to_maturity',
                                                   'fixing_frequency',
                                                   'past_fixings',
                                                   'risk_free_rate',
                                                   'dividend_rate', 'kappa',
                                                   'theta', 'rho', 'eta', 'v0',
                                                   'relative_spot']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['averaging_type', 'w'])])),
                ('regressor',
                 MLPRegressor(early_stopping=True, hidden_layer_sizes=(20,),
                              learning_rate='adaptive', learning_rate_init=0.1,
                              max_iter=500, n_iter_no_change=20,
                              random_state=1312, solver='sgd'))])

In [7]:
test_performance(model,net)

out of sample
   RMSE:
      0.005064355865830351
   MAE:
      0.0030570028051901126
out of sample
   RMSE:
      0.055429415545038066
   MAE:
      0.03542515603491769


In [8]:
import pandas as pd
param_df = pd.DataFrame()
for k,v in net[1].__dict__.items():
    try:
        param_df.loc[str(k),'specification'] = str(v)
    except Exception:
        pass

param_df = param_df[:31].copy()
param_df = param_df.drop(index=[i for i in param_df.index if i.startswith('_')]).reset_index()
param_df = param_df.rename(columns={'index':'parameter'})
latex_table = param_df.to_latex(index=False, escape=False)
print(latex_table)

\begin{tabular}{ll}
\toprule
parameter & specification \\
\midrule
activation & relu \\
solver & sgd \\
alpha & 0.0001 \\
batch_size & auto \\
learning_rate & adaptive \\
learning_rate_init & 0.1 \\
power_t & 0.5 \\
max_iter & 500 \\
loss & squared_error \\
hidden_layer_sizes & (20,) \\
shuffle & True \\
random_state & 1312 \\
tol & 0.0001 \\
verbose & False \\
warm_start & False \\
momentum & 0.9 \\
nesterovs_momentum & True \\
early_stopping & True \\
validation_fraction & 0.1 \\
beta_1 & 0.9 \\
beta_2 & 0.999 \\
epsilon & 1e-08 \\
n_iter_no_change & 20 \\
max_fun & 15000 \\
out_activation_ & identity \\
n_features_in_ & 15 \\
n_outputs_ & 1 \\
n_iter_ & 196 \\
t_ & 4159512 \\
n_layers_ & 3 \\
\bottomrule
\end{tabular}

